In [ ]:
es_index = "./es-indices.txt"

In [ ]:
import pandas as pd

pd.set_option('display.width', 1000)

In [ ]:
import numpy as np
from matplotlib import pyplot as plt, rcParams

rcParams['font.family'] = 'VL Gothic'
rcParams['font.weight'] = 'regular'
rcParams['axes.titlesize'] = 18
rcParams['xtick.labelsize'] = 14
rcParams['ytick.labelsize'] = 14


In [ ]:
import re
def normalize(size):
    size, *unit = re.findall(r'([0-9.]+|\D+)',size)
    if (unit == 'kb'):
       size = int(float(size) * 1024)
    elif (unit == 'mb'):
       size = int(float(size) * 1024 * 1024)
    elif (unit == 'gb'):
       size = int(float(size) * 1024 * 1024 * 1024)
    return size #/ 1024 / 1024


In [ ]:
df = pd.read_csv(es_index, delim_whitespace=True)
sizedf = pd.concat([df[['index', 'store.size']],df['index'].str.extract('(?P<index_series>.*)-(?P<date>\d{4}.\d{2}.\d{2})', expand=True)], axis=1)
sizedf['store.size'] = sizedf['store.size'].map(normalize)
sizedf['date'] = pd.to_datetime(sizedf['date'])
sizedf.dropna(inplace=True)

In [ ]:
index_df = pd.DataFrame(sizedf.drop_duplicates('date').sort_values(by=['date'])['date'].reset_index(drop=True))
for lindex in sizedf['index_series'].drop_duplicates():
    index_df = pd.merge(index_df, sizedf[sizedf.index_series == lindex][['date','store.size']].rename(columns={'store.size':lindex}), on='date', how='outer')

In [ ]:
fig = plt.figure(figsize=(20,8))
ax = fig.add_subplot(111)

for lindex in np.delete(index_df.columns.values, 0):
    ax.bar(index_df['date'],index_df[lindex])
plt.style.use('dark_background')
plt.show()
